## Model Building

Notebook context: Snippets of python script in building a baseline model and performing a grid search


The objective of this notebook is to summarize the following


 + Baseline Model (AutoML from H2o)
 + GridSearch (H2o)
 + Bayesian Optimization GridSearch (hyperopt)
 
#### Dataset to download

https://www.kaggle.com/c/allstate-claims-severity


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pylab as plt
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from hyperopt import hp, tpe, STATUS_OK, Trials, fmin
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()
# h2o.cluster().shutdown()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 0 hours 50 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_saiv_6gvmi4
H2O cluster total nodes:,1
H2O cluster free memory:,2.124 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


#### Import a sample binary outcome train/test set into H2O 

In [2]:
data = h2o.import_file("/Users/saiv/.kaggle/competitions/allstate-claims-severity/train.csv.zip")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
data.describe()

Rows:188318
Cols:132




,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
type,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,1.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1.6e-05,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.23688,8e-05,0.0,0.035321,0.036232,0.000228,0.179722,0.67
mean,294135.982561412,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.4938613645641946,0.5071883561794411,0.4989184507216516,0.4918123025892374,0.4874277287832284,0.49094453373548974,0.4849702050680232,0.4864373158699645,0.4855063198950708,0.4980658504232198,0.493511008554679,0.493150425625803,0.49313761583597965,0.4957170179749147,3037.3376856699833
maxs,587633.0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.0,0.9802,0.9954,0.99498,0.998742,0.998484,0.988494,0.844848,121012.25
sigma,169336.08486658742,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.18764017641388622,0.20720173860981372,0.2021046081934374,0.21129221269283555,0.20902682854450408,0.20527256983553027,0.17845016396070926,0.19937045456133262,0.18166017135075618,0.18587672593201837,0.20973651144747818,0.20942662107602883,0.21277724232240963,0.2224875395592257,2904.086186390402
zeros,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,225,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.7263,0.245921,0.187583,0.789639,0.310061,0.718367,0.33506,0.3026,0.67135,0.8351,0.569745,0.594646,0.822493,0.714843,2213.18
1,2.0,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.6
2,5

In [4]:
target = 'loss'
ID = 'id'
data.drop([ID], axis=1)
train, valid = data.split_frame(ratios=[0.8], seed=1)
# valid = data.drop(train.index)
features = train.columns

In [5]:
cats = [feat for feat in features if 'cat' in feat]
for feat in cats:
    train[feat] = train[feat].asfactor()
    valid[feat] = valid[feat].asfactor()

y_train = (train[target])
y_valid = (valid[target])

# y_train = np.log(train[target].ravel())
# y_valid = np.log(valid[target].ravel())

In [6]:
# Identify predictors and response
x = train.columns

aml = H2OAutoML(max_runtime_secs = 30)
aml.train(x = x, y = "loss",
          training_frame = train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_0_AutoML_20180907_141644,4.23395e+06,2057.66,4.23395e+06,1277.5,0.599227


If you need to generate predictions on a test set, you can make predictions directly on the `"H2OAutoML"` object, or on the leader model object directly

In [7]:
# The leader model is stored here
aml.leader
preds = aml.predict(valid)
# or:
preds = aml.leader.predict(valid)

drf prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [ ]:
# GBM hyperparameters

gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y="loss",
                training_frame=train,
                validation_frame=valid,
                ntrees=100,
                seed=1)

# Get the grid results, sorted by validation MAE
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='mae', decreasing=False)

gbm Grid Build progress: |██████████████████

### Grid Search with H2o

In [ ]:
gbm_gridperf1

Now let's evaluate the model performance on a test set so we get an honest estimate of top model performance

In [ ]:
# Grab the top GBM model, chosen by validation MAE
best_gbm1 = gbm_gridperf1.models[0]
best_gbm_perf1 = best_gbm1.model_performance(valid)
best_gbm_perf1

### Hyperparameter with hyperopt

In [ ]:
def xgb_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

def objective(space):

    clf = xgb.XGBRegressor(n_estimators = space['n_estimators'],
                           max_depth = space['max_depth'],
                           min_child_weight = space['min_child_weight'],
                           subsample = space['subsample'],
                           learning_rate = space['learning_rate'],
                           gamma = space['gamma'],
                           colsample_bytree = space['colsample_bytree'],
                           objective='reg:linear',
                           early_stopping_rounds = 10
                           )

    eval_set  = [( train, y_train), ( valid, y_valid)]

    clf.fit(train[col_train],
            y_train,
            eval_set=eval_set,
            eval_metric = 'mae')
    pred = clf.predict(valid)
    #   mae = mean_absolute_error(np.exp(y_valid), np.exp(pred))
    mae = mean_absolute_error((y_valid), (pred))

    #    print "SCORE:", mae
    return{'loss':mae, 'status': STATUS_OK }


space = {
        'max_depth': hp.choice('max_depth', np.arange(10, 30, dtype=int)),
        'min_child_weight': hp.quniform ('min_child', 1, 20, 1),
        'subsample': hp.uniform ('subsample', 0.8, 1),
        'n_estimators' : hp.choice('n_estimators', np.arange(1000, 10000, 100, dtype=int)),
        'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05)
    }

In [ ]:
train = train.as_data_frame()
valid = valid.as_data_frame()


features = train.columns

cats = [feat for feat in features if 'cat' in feat]
for feat in cats:
    train[feat] = pd.factorize(train[feat], sort=True)[0]
    valid[feat] = pd.factorize(valid[feat], sort=True)[0]


y_train = (train[target])
y_valid = (valid[target])

#y_train = np.log(train[target].ravel())
#y_valid = np.log(valid[target].ravel())

del train[target]
del valid[target]

col_train = train.columns

trials = Trials()

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals = 100, # change
            trials=trials)



### Results Comparison

Basemodel MAE: 1280
GridSearch MAE:
HyperOpt MAE: